# Training on a single node

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, add


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### Define a model

In [8]:
def toy_resnet():
    inputs = Input(shape=(32, 32, 3), name='image')
    x = Conv2D(32, 3, activation='relu')(inputs)
    x = Conv2D(64, 3, activation='relu')(x)
    block_1_output = MaxPooling2D(3)(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    block_2_output = add([x, block_1_output])
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    block_3_output = add([x, block_2_output])
    
    x = Conv2D(64, 3, activation='relu')(block_3_output)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='toy_resnet')
    
    return model

model = toy_resnet()

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 30, 30, 32)   896         image[0][0]                      
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 28, 28, 64)   18496       conv2d_7[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 9, 9, 64)     0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (

### Plot the model

In [ ]:
tf.keras.utils.plot_model(model, 'toy_resnet.png', show_shapes=True)

In [39]:
def load_data():
    cifar10_builder = tfds.builder("cifar10")
    cifar10_builder.download_and_prepare()
    cifar10_train, cifar10_valid = cifar10_builder.as_dataset(split=[tfds.Split.TRAIN, tfds.Split.TEST], as_supervised=True)
    
    return cifar10_train, cifar10_valid
    

In [49]:
train_dataset, val_dataset = load_data()
print(train_dataset)

<DatasetV1Adapter shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>


In [50]:
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255
    
    return image, label

train_dataset = train_dataset.map(preprocess_image)
val_dataset = val_dataset.map(preprocess_image)
train_dataset = train_dataset.shuffle(4096).batch(32).repeat()
val_dataset = val_dataset.batch(32).repeat()

In [51]:
print(train_dataset)

<DatasetV1Adapter shapes: ((?, 32, 32, 3), (?,)), types: (tf.float32, tf.int64)>


In [53]:


model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3),
             loss="sparse_categorical_crossentropy",
             metrics=["sparse_categorical_accuracy"]
             )

model.fit(train_dataset,
         epochs=20,
         steps_per_epoch=1000,
         validation_data=val_dataset,
         validation_steps=200)


Epoch 1/20
1000/1000 [==============================] - 21s 21ms/step - loss: 1.2661 - sparse_categorical_accuracy: 0.5481 - val_loss: 1.0766 - val_sparse_categorical_accuracy: 0.6158
Epoch 2/20
1000/1000 [==============================] - 18s 18ms/step - loss: 1.0926 - sparse_categorical_accuracy: 0.6158 - val_loss: 1.1486 - val_sparse_categorical_accuracy: 0.5874
Epoch 3/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.9877 - sparse_categorical_accuracy: 0.6552 - val_loss: 0.8918 - val_sparse_categorical_accuracy: 0.6816
Epoch 4/20
1000/1000 [==============================] - 18s 18ms/step - loss: 0.9142 - sparse_categorical_accuracy: 0.6847 - val_loss: 0.8478 - val_sparse_categorical_accuracy: 0.7027
Epoch 5/20
1000/1000 [==============================] - 18s 18ms/step - loss: 0.8652 - sparse_categorical_accuracy: 0.7046 - val_loss: 0.8448 - val_sparse_categorical_accuracy: 0.7144
Epoch 6/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.

In [54]:
from absl import flags

### Create a training script

In [ ]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

In [55]:
SCRIPT_FOLDER = './training'

In [56]:
!mkdir $SCRIPT_FOLDER

In [57]:
!touch $SCRIPT_FOLDER/__init__.py

In [85]:
%%writefile $SCRIPT_FOLDER/training.py

from absl import flags
from absl import app

import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, add

tf.enable_eager_execution()


IMAGE_SHAPE = (32, 32, 3)
NUM_CLASSES = 10

def toy_resnet_model():
    inputs = Input(shape=IMAGE_SHAPE, name='image')
    x = Conv2D(32, 3, activation='relu')(inputs)
    x = Conv2D(64, 3, activation='relu')(x)
    block_1_output = MaxPooling2D(3)(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    block_2_output = add([x, block_1_output])
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    block_3_output = add([x, block_2_output])
    
    x = Conv2D(64, 3, activation='relu')(block_3_output)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='toy_resnet')
    
    return model


def prepare_datasets():
    def _parse_record(example_proto):
        features = {
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64, default_value=0)
        }
        
        parsed_features = tf.parse_single_example(example_proto, features)
        image = parsed_features['image']
        label = parsed_features['label']
        
        image = tf.image.decode_png(image, channels=3)
        #image = tf.cast(image, tf.float32)
        #image = image / 255
        #image = tf.reshape(image, IMAGE_SHAPE)
        
        #label = tf.one_hot(label, NUM_CLASSES)
        
        #return image, label
        return image, label

    training_images = [os.path.join(FLAGS.training_images, file) for file in os.listdir(FLAGS.training_images)]
    validation_images = [os.path.join(FLAGS.validation_images, file) for file in os.listdir(FLAGS.validation_images)]
    
    train_dataset = tf.data.TFRecordDataset(training_images)
    val_dataset = tf.data.TFRecordDataset(validation_images)
    
    train_dataset = train_dataset.map(_parse_record)
    val_dataset = val_dataset.map(_parse_record)
    
    train_dataset = train_dataset.shuffle(4096).batch(FLAGS.batch_size).repeat()
    val_dataset = val_dataset.batch(FLAGS.batch_size).repeat()
    
    return train_dataset, val_dataset


def train_evaluate():
    
    train_dataset, val_dataset = prepare_datasets()
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3),
             loss="sparse_categorical_crossentropy",
             metrics=["sparse_categorical_accuracy"]
             )

    model.fit(train_dataset,
         epochs=20,
         steps_per_epoch=1000,
         validation_data=val_dataset,
         validation_steps=200)
    
    

FLAGS = flags.FLAGS
flags.DEFINE_string("training_images", None, "Training images")
flags.DEFINE_string("validation_images", None, "Validation images")
flags.DEFINE_integer("epochs", 10, "Number of epochs to train")
flags.DEFINE_integer("batch_size", 1, "Batch size")

# Required flags
flags.mark_flag_as_required("training_images")
flags.mark_flag_as_required("validation_images")

def main(argv):
    del argv #Unused
    
    print(FLAGS.training_images)
    print(FLAGS.validation_images)
    
    train, val = prepare_datasets()
    
    for record in train.take(1):
        print(record)
    

if __name__ == '__main__':
    
    app.run(main)


Overwriting ./training/training.py
